In [1]:
import os

import aiohttp
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv(".env")


True

In [3]:
from lmi import CommonLLMNames

llm_openai = CommonLLMNames.OPENAI_TEST.value
llm_anthropic = CommonLLMNames.ANTHROPIC_TEST.value

# Create the `papers` directory if it doesn't exist
os.makedirs("papers", exist_ok=True)

# Download the paper from arXiv and save it to the `papers` directory
url = "https://arxiv.org/pdf/2407.01603"
async with aiohttp.ClientSession() as session, session.get(url, timeout=60) as response:
    content = await response.read()
    with open("papers/2407.01603.pdf", "wb") as f:
        f.write(content)

In [6]:
import pathlib

from paperqa.prompts import (
    CONTEXT_INNER_PROMPT,
    CONTEXT_OUTER_PROMPT,
    citation_prompt,
    default_system_prompt,
    env_reset_prompt,
    env_system_prompt,
    qa_prompt,
    select_paper_prompt,
    structured_citation_prompt,
    summary_json_prompt,
    summary_json_system_prompt,
    summary_prompt,
)
from paperqa.settings import (
    AgentSettings,
    AnswerSettings,
    IndexSettings,
    ParsingSettings,
    PromptSettings,
    Settings,
)

settings = Settings(
    llm=llm_openai,
    llm_config={
        "model_list": [
            {
                "model_name": llm_openai,
                "litellm_params": {
                    "model": llm_openai,
                    "temperature": 0.1,
                    "max_tokens": 4096,
                },
            }
        ],
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    summary_llm=llm_openai,
    summary_llm_config={
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    embedding="text-embedding-3-small",
    embedding_config={},
    temperature=0.1,
    batch_size=1,
    verbosity=1,
    manifest_file=None,
    paper_directory=pathlib.Path.cwd().joinpath("papers"),
    index_directory=pathlib.Path.cwd().joinpath("papers/index"),
    answer=AnswerSettings(
        evidence_k=10,
        evidence_detailed_citations=True,
        evidence_retrieval=True,
        evidence_summary_length="about 100 words",
        evidence_skip_summary=False,
        answer_max_sources=5,
        max_answer_attempts=None,
        answer_length="about 200 words, but can be longer",
        max_concurrent_requests=10,
    ),
    parsing=ParsingSettings(
        chunk_size=5000,
        overlap=250,
        citation_prompt=citation_prompt,
        structured_citation_prompt=structured_citation_prompt,
    ),
    prompts=PromptSettings(
        summary=summary_prompt,
        qa=qa_prompt,
        select=select_paper_prompt,
        pre=None,
        post=None,
        system=default_system_prompt,
        use_json=True,
        summary_json=summary_json_prompt,
        summary_json_system=summary_json_system_prompt,
        context_outer=CONTEXT_OUTER_PROMPT,
        context_inner=CONTEXT_INNER_PROMPT,
    ),
    agent=AgentSettings(
        agent_llm=llm_openai,
        agent_llm_config={
            "model_list": [
                {
                    "model_name": llm_openai,
                    "litellm_params": {
                        "model": llm_openai,
                    },
                }
            ],
            "rate_limit": {
                llm_openai: "30000 per 1 minute",
            },
        },
        agent_prompt=env_reset_prompt,
        agent_system_prompt=env_system_prompt,
        search_count=8,
        index=IndexSettings(
            paper_directory=pathlib.Path.cwd().joinpath("papers"),
            index_directory=pathlib.Path.cwd().joinpath("papers/index"),
        ),
    ),
)

In [7]:
from paperqa import ask

response = ask(
    "What are the most relevant language models used for chemistry?", settings=settings
)

PaperQA version: 5.20.0


[22:35:21] Beginning agent 'ToolSelector' run with question 'What are the most relevant language models used for   
           chemistry?' and full settings {'llm': 'gpt-4o-mini-2024-07-18', 'llm_config': {'model_list':            
           [{'model_name': 'gpt-4o-mini-2024-07-18', 'litellm_params': {'model': 'gpt-4o-mini-2024-07-18',         
           'temperature': 0.1, 'max_tokens': 4096}}], 'rate_limit': {'gpt-4o-mini-2024-07-18': '30000 per 1        
           minute'}}, 'summary_llm': 'gpt-4o-mini-2024-07-18', 'summary_llm_config': {'rate_limit':                
           {'gpt-4o-mini-2024-07-18': '30000 per 1 minute'}}, 'embedding': 'text-embedding-3-small',               
           'embedding_config': {}, 'temperature': 0.1, 'batch_size': 1, 'texts_index_mmr_lambda': 1.0, 'verbosity':
           1, 'answer': {'evidence_k': 10, 'evidence_detailed_citations': True, 'evidence_retrieval': True,        
           'evidence_summary_length': 'about 100 words', 'evidence_skip_summary': False, 'answer_max_sources': 5,  
           'max_answer_attempts': None, 'answer_length': 'about 200 words, but can be longer',                     
           'max_concurrent_requests': 10, 'answer_filter_extra_background': False, 'get_evidence_if_no_contexts':  
           True}, 'parsing': {'chunk_size': 5000, 'page_size_limit': 1280000, 'use_doc_details': True, 'overlap':  
           250, 'citation_prompt': 'Provide the citation for the following text in MLA Format. Do not write an     
           introductory sentence. If reporting date accessed, the current year is 2025\n\n{text}\n\nCitation:',    
           'structured_citation_prompt': "Extract the title, authors, and doi as a JSON from this MLA citation. If 
           any field can not be found, return it as null. Use title, authors, and doi as keys, author's value      
           should be a list of authors. {citation}\n\nCitation JSON:", 'disable_doc_valid_check': False,           
           'defer_embedding': False, 'chunking_algorithm': <ChunkingOptions.SIMPLE_OVERLAP: 'simple_overlap'>,     
           'doc_filters': None, 'use_human_readable_clinical_trials': False}, 'prompts': {'summary': 'Summarize the
           excerpt below to help answer a question.\n\nExcerpt from                                                
           {citation}\n\n----\n\n{text}\n\n----\n\nQuestion: {question}\n\nDo not directly answer the question,    
           instead summarize to give evidence to help answer the question. Stay detailed; report specific numbers, 
           equations, or direct quotes (marked with quotation marks). Reply "Not applicable" if the excerpt is     
           irrelevant. At the end of your response, provide an integer score from 1-10 on a newline indicating     
           relevance to question. Do not explain your score.\n\nRelevant Information Summary ({summary_length}):', 
           'qa': 'Answer the question below with the context.\n\nContext (with relevance                           
           scores):\n\n{context}\n\n----\n\nQuestion: {question}\n\nWrite an answer based on the context. If the   
           context provides insufficient information reply "I cannot answer." For each part of your answer,        
           indicate which sources most support it via citation keys at the end of sentences, like                  
           {example_citation}. Only cite from the context below and only use the valid keys. Write in the style of 
           a Wikipedia article, with concise sentences and coherent paragraphs. The context comes from a variety of
           sources and is only a summary, so there may inaccuracies or ambiguities. If quotes are present and      
           relevant, use them in the answer. This answer will go directly onto Wikipedia, so do not add any        
           extraneous information.\n\n{prior_answer_prompt}Answer ({answer_length}):', 'answer_iteration_prompt':  
           'You are iterating on a prior answer, with a 

           New file to index: 2407.01603.pdf...

[22:35:25] SEMANTIC_SCHOLAR_API_KEY environment variable not set. Semantic Scholar API rate limits may apply.

           CROSSREF_MAILTO environment variable not set. Crossref API rate limits may apply.

...

           Generating answer for 'What are the most relevant language models used for chemistry?'.

[22:35:55] Status: Paper Count=1 | Relevant Papers=1 | Current Evidence=10 | Current Cost=$0.0038

[22:35:56] Completing 'What are the most relevant language models used for chemistry?' as 'certain'.

           Finished agent 'ToolSelector' run with question 'What are the most relevant language models used for    
           chemistry?' and status success.

           Answer: Several language models have emerged as significant tools in the field of chemistry,            
           particularly for molecular property prediction and design. Notable models include Text2Mol, which       
           connects natural language descriptions with molecular representations, and MolT5, which generates       
           molecular captions from Simplified Molecular Input Line Entry System (SMILES) and predicts molecular    
           structures from textual descriptions (Caldas2024 pages 23-24).                                          
                                                                                                                   
           Other important models are CLAMP, which predicts biochemical activity, and BioTranslator, which         
           translates text into biological data (Caldas2024 pages 23-24). Additionally, Cappy enhances large       
           multi-task language models, while iPaucGPT is specifically designed for molecular property prediction   
           and generation (Caldas2024 pages 50-51). Tag-LLM repurposes general-purpose models for specialized      
           domains, and NExT-GPT is noted for its multimodal capabilities (Caldas2024 pages 50-51).                
                                                                                                                   
           In the realm of materials science, models like MaterialBERT and SolvBERT focus on natural language      
           processing and predicting solvation free energy, respectively (Caldas2024 pages 55-56). Furthermore,    
           attention-based neural networks and SMILES-BERT are utilized for mapping chemical reactions and         
           predicting molecular properties (Caldas2024 pages 58-58). These models collectively enhance the         
           understanding and prediction of chemical behaviors and properties in various applications.